# Load Libraries

In [1]:
%matplotlib inline
import PetThermoTools as M
import numpy as np
import pandas as pd

In [2]:
# check version of PetThermoTools 
print(M.__version__)

0.2.1


## Modelling fractional crystallisation paths using RhyoliteMELTS run through PetThermoTools 

In [3]:
# Read in desired file to be input 
file = pd.read_csv('PetThermo_input.csv')
header = file.columns.values.tolist()
file

P  Temp  FMQ  CO2  SiO2  TiO2  Al2O3     FeO   Fe2O3    MgO  ...  Na2O  \
0  2000  1350    0    0  49.2  0.69   12.9  7.2689  2.6907  12.93  ...  1.99   

    K2O  MnO  P2O5  H2O  Cr2O5  NiO  Ni  Cu    FeOt  
0  0.97  0.2   0.2    1      0    0  20  30  9.9596  

[1 rows x 21 columns]

In [4]:
bulk = file.iloc[0].values.tolist()

h2o = [0,0.1,1.0,3.0,6.0]
buff = pd.DataFrame()

for i in range(len(h2o)):
    bulk[15] = h2o[i]
    buff = pd.concat([buff, pd.DataFrame([bulk])], ignore_index=True)

buff.columns = header
buff = buff[['SiO2', 'TiO2', 'Al2O3','FeOt', 'MgO', 'CaO', 'Na2O', 'K2O','MnO', 'P2O5', 'H2O']]
buff['QFM']=  1.2
buff.columns = [str(col) + '_Liq' for col in buff.columns]
buff

SiO2_Liq  TiO2_Liq  Al2O3_Liq  FeOt_Liq  MgO_Liq  CaO_Liq  Na2O_Liq  \
0      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   
1      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   
2      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   
3      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   
4      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   

   K2O_Liq  MnO_Liq  P2O5_Liq  H2O_Liq  QFM_Liq  
0     0.97      0.2       0.2      0.0      1.2  
1     0.97      0.2       0.2      0.1      1.2  
2     0.97      0.2       0.2      1.0      1.2  
3     0.97      0.2       0.2      3.0      1.2  
4     0.97      0.2       0.2      6.0      1.2

In [6]:
# Creating column indexer 
buff['buffer'] = 'buffered'
buff['dictindex'] = 'H2O_' + buff['H2O_Liq'].astype(str)  + '_' 'QFM' + buff['QFM_Liq'].astype(str) #+ '_' 'CO2_' + merged['CO2_Liq'].astype(str)
buff['H2O_bulk'] = buff['H2O_Liq']
buff

SiO2_Liq  TiO2_Liq  Al2O3_Liq  FeOt_Liq  MgO_Liq  CaO_Liq  Na2O_Liq  \
0      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   
1      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   
2      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   
3      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   
4      49.2      0.69       12.9    9.9596    12.93    11.21      1.99   

   K2O_Liq  MnO_Liq  P2O5_Liq  H2O_Liq  QFM_Liq    buffer       dictindex  \
0     0.97      0.2       0.2      0.0      1.2  buffered  H2O_0.0_QFM1.2   
1     0.97      0.2       0.2      0.1      1.2  buffered  H2O_0.1_QFM1.2   
2     0.97      0.2       0.2      1.0      1.2  buffered  H2O_1.0_QFM1.2   
3     0.97      0.2       0.2      3.0      1.2  buffered  H2O_3.0_QFM1.2   
4     0.97      0.2       0.2      6.0      1.2  buffered  H2O_6.0_QFM1.2   

   H2O_bulk  
0       0.0  
1       0.1  
2       1.0  
3       3.0  
4       6.0

In [7]:
P_lin = ([10000.0,8000.0,4000.0,2000.0,1000.0,500.0])
P_results_buff = {}
for i in range(len(P_lin)):
    P_bar = P_lin[i]
    print(P_bar)
    Results =  M.isobaric_crystallisation(Model = "MELTSv1.2.0",
                                           bulk = buff,
                                           find_liquidus = True,
                                           P_bar = P_bar,
                                           T_end_C = 800,
                                           dt_C = 5,
                                           fO2_buffer = "FMQ",
                                           fO2_offset = 1.2,
                                           Frac_solid = True,
                                           Frac_fluid = True,
                                           H2O_Liq = None,
                                           CO2_Liq = None)#
    
    P_results_buff['P = ' + str(P_lin[i]) + ' bars'] = Results
    
if not P_results_buff:  
    print('You do n0t seem to have alphaMELTS for python installed correctly - your calculation has returned an empty dictionary. You need to add the path to MELTS here ')

10000.0


Running MELTSv1.2.0 calculations 0 to 4.0 ... Complete (time taken = 16.81 seconds)

dict_keys(['index = 1', 'index = 0', 'index = 4', 'index = 3', 'index = 2'])
8000.0


Running MELTSv1.2.0 calculations 0 to 4.0 ... Complete (time taken = 17.73 seconds)

dict_keys(['index = 0', 'index = 4', 'index = 3', 'index = 1', 'index = 2'])
4000.0


Running MELTSv1.2.0 calculations 0 to 4.0 ... Complete (time taken = 15.47 seconds)

dict_keys(['index = 0', 'index = 4', 'index = 3', 'index = 2', 'index = 1'])
2000.0


Running MELTSv1.2.0 calculations 0 to 4.0 ... Complete (time taken = 11.49 seconds)

dict_keys(['index = 3', 'index = 0', 'index = 4', 'index = 2', 'index = 1'])
1000.0


Running MELTSv1.2.0 calculations 0 to 4.0 ... Complete (time taken = 260.21 seconds)

dict_keys(['index = 0', 'index = 3', 'index = 2', 'index = 4'])
500.0


Running MELTSv1.2.0 calculations 0 to 4.0 ... Complete (time taken = 260.15 seconds)

dict_keys(['index = 0', 'index = 4', 'index = 3', 'index = 2'])


In [8]:
# check to see that all pressures ran successfully:
P_results_buff.keys()

dict_keys(['P = 10000.0 bars', 'P = 8000.0 bars', 'P = 4000.0 bars', 'P = 2000.0 bars', 'P = 1000.0 bars', 'P = 500.0 bars'])

In [9]:
# convert nested dict to a dataframe ready to be used for the rest of our model
empty = pd.DataFrame() 

for outer_key in P_results_buff:
    # p_dict is a dictionary of a single pressure 
    p_dict = P_results_buff[outer_key]
    print('outerkey= ',outer_key)
    final_dict = p_dict
    for key in final_dict:
 
        new = final_dict[key]['All']
        new['dictindex'] = key
        new[['pressure_bar', 'T_C']] = final_dict[key]['Conditions'][['P_bar','T_C']]
        print(new['pressure_bar'])
        #empty = empty.append(new) deprecated
        empty = pd.concat([empty, new], ignore_index=True)

outerkey=  P = 10000.0 bars
0      10000.0
1      10000.0
2      10000.0
3      10000.0
4      10000.0
        ...   
120        NaN
121        NaN
122        NaN
123        NaN
124        NaN
Name: pressure_bar, Length: 125, dtype: float64
0      10000.0
1      10000.0
2      10000.0
3      10000.0
4      10000.0
        ...   
121        NaN
122        NaN
123        NaN
124        NaN
125        NaN
Name: pressure_bar, Length: 126, dtype: float64
0     10000.0
1     10000.0
2     10000.0
3     10000.0
4     10000.0
       ...   
86    10000.0
87    10000.0
88    10000.0
89    10000.0
90    10000.0
Name: pressure_bar, Length: 91, dtype: float64
0      10000.0
1      10000.0
2      10000.0
3      10000.0
4      10000.0
        ...   
96     10000.0
97     10000.0
98     10000.0
99     10000.0
100    10000.0
Name: pressure_bar, Length: 101, dtype: float64
0      10000.0
1      10000.0
2      10000.0
3      10000.0
4      10000.0
        ...   
111    10000.0
112    10000.0
113    10000

In [10]:
# check NaNs and shape
empty.pressure_bar.unique(), empty.shape

(array([10000.,    nan,  8000.,  4000.,  2000.,  1000.,   500.]), (2836, 369))

In [11]:
empty['pressure_bar'] = pd.to_numeric(empty['pressure_bar'], errors='coerce')
empty.dropna(subset=['pressure_bar'], inplace=True)

In [12]:
# check NaNs and shape
print( 'After coercing data:', empty.pressure_bar.unique(), empty.shape)

After coercing data: [10000.  8000.  4000.  2000.  1000.   500.] (2445, 369)


In [13]:
# replace 'dictindex' values with H2O_QFM identifiers
buff['index'] = buff.index 
newkeys = buff[['index','dictindex']]
empty_index = empty.reset_index()

In [14]:
empty_index['newentry'] = empty_index['dictindex'].str.extract('(\d+)')

In [15]:
empty_index['newcolumn'] = 0
for i in range(len(empty_index)):
    
    empty_index['newentry'][i] = float(empty_index['newentry'][i])
    empty_index['newcolumn'][i] = newkeys['dictindex'][empty_index['newentry'][i]]
empty_index.shape

<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newcolumn'][i] = newkeys['dictindex'][empty_index['newentry'][i]]
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newent

<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipytho

<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipytho

<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipytho

<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipython-input-15-2c1f78c14217>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  empty_index['newentry'][i] = float(empty_index['newentry'][i])
<ipytho

(2445, 372)

In [16]:
empty_index['unique'] = 'P_'+ empty_index['P_bar'].astype(str) + '_' + empty_index['newcolumn'].astype(str)
empty = empty_index.copy()

In [17]:
empty_index['unique'].unique()

array(['P_10000.0_H2O_0.1_QFM1.2', 'P_10000.0_H2O_0.0_QFM1.2',
       'P_10000.0_H2O_6.0_QFM1.2', 'P_10000.0_H2O_3.0_QFM1.2',
       'P_10000.0_H2O_1.0_QFM1.2', 'P_8000.0_H2O_0.0_QFM1.2',
       'P_8000.0_H2O_6.0_QFM1.2', 'P_8000.0_H2O_3.0_QFM1.2',
       'P_8000.0_H2O_0.1_QFM1.2', 'P_8000.0_H2O_1.0_QFM1.2',
       'P_4000.0_H2O_0.0_QFM1.2', 'P_4000.0_H2O_6.0_QFM1.2',
       'P_4000.0_H2O_3.0_QFM1.2', 'P_4000.0_H2O_1.0_QFM1.2',
       'P_4000.0_H2O_0.1_QFM1.2', 'P_2000.0_H2O_3.0_QFM1.2',
       'P_2000.0_H2O_0.0_QFM1.2', 'P_2000.0_H2O_6.0_QFM1.2',
       'P_2000.0_H2O_1.0_QFM1.2', 'P_2000.0_H2O_0.1_QFM1.2',
       'P_1000.0_H2O_0.0_QFM1.2', 'P_1000.0_H2O_3.0_QFM1.2',
       'P_1000.0_H2O_1.0_QFM1.2', 'P_1000.0_H2O_6.0_QFM1.2',
       'P_500.0_H2O_0.0_QFM1.2', 'P_500.0_H2O_6.0_QFM1.2',
       'P_500.0_H2O_3.0_QFM1.2', 'P_500.0_H2O_1.0_QFM1.2'], dtype=object)

In [18]:
# export dataframe
empty.to_excel('PetThermo_output.xlsx')